In [ ]:
import yfinance
from math import exp, log, pi, sqrt
import py_vollib.black_scholes
import pandas as pd
import numpy as np
import datetime as dt
import py_vollib_vectorized
from py_vollib_vectorized import price_dataframe, get_all_greeks
import random
from pyecharts.charts import Bar, HeatMap
from pyecharts import options as opts

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
pd.options.display.float_format = '{:.5f}'.format



def generate_date_sequence(start_date, end_date, delta_days=1):
    """
    Generate a sequence of dates from start_date to end_date with a step of delta_days.
    
    :param start_date: A datetime.date object indicating the start date.
    :param end_date: A datetime.date object indicating the end date.
    :param delta_days: The number of days to step through (default is 1 for daily).
    :return: A list of datetime.date objects.
    """
    dates = []
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += dt.timedelta(days=delta_days)


#get option chain from yfinance
tsla = yfinance.Ticker('TSLA')
tsla.options
stock_price = tsla.history('1day')['Close'].iloc[0]
calls = tsla.option_chain('2024-02-09').calls
calls
remaining_days = dt.datetime(2024,2,9) - dt.datetime.today()
remaining_days.days

start = dt.datetime.today().date()  # Start date: January 1, 2023
end = dt.datetime(2024, 2, 9).date()   # End date: January 10, 2023

all_dates = list(generate_date_sequence(start, end))
all_dates_str = [i.strftime('%Y-%m-%d') for i in all_dates]
print(all_dates_str)

#calculate option prices
df_total = pd.DataFrame()
for i in range(remaining_days.days, -1, -1):
    df = pd.DataFrame({'Flag': 'c',
                     'S': [i for i in range(170,190,1)],
                     'K':185,
                     'T': i/252,
                     'R':0.05,
                    'IV':0.43818}) #calls.loc[calls['strike'] == 185, 'impliedVolatility'].iloc[0],})
    result = price_dataframe(df, flag_col='Flag', underlying_price_col='S', strike_col='K', annualized_tte_col='T',
                     riskfree_rate_col='R', sigma_col='IV', model='black_scholes', inplace=False)
    price = result['Price']
    df_total[i] = round(price,2)
df_total['stock_price'] = df['S']
df_pnl = df_total.copy()
for i in range(remaining_days.days, -1, -1):
    df_pnl[i] = round((df_pnl[i])*100,0)
#df_pnl.sort_values('stock_price', ascending=False, inplace=True)
new_cols = [i for i in range(remaining_days.days+1)]
new_cols.append('stock_price')
df_pnl.columns = new_cols
df_pnl

#convert price df to list
value = []
rows = df_pnl.shape[0]
for i in range(remaining_days.days,-1, -1):
#for i in range(0,remaining_days.days+1):
    for ind, j in enumerate(df_pnl[i]):
        value.append([i,ind, j])
        
c = (
    HeatMap(init_opts=opts.InitOpts(width='800px', height='800px'))
    .add_xaxis(all_dates_str)
    .extend_axis(
    xaxis_data=all_dates_str,
    xaxis=opts.AxisOpts(position="top"))
    .add_yaxis(
        "series0",
        df_pnl['stock_price'].tolist(),
        value,
        label_opts=opts.LabelOpts(is_show=True, position="inside"), #, color='black'
    )
    .set_global_opts(
        # xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(color="black", width=2))),
        # yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True,linestyle_opts=opts.LineStyleOpts(color="black", width=2))),
        title_opts=opts.TitleOpts(title="option profits"),
        visualmap_opts=opts.VisualMapOpts(max_=450, min_=-560,range_color=["red","white", "lime"], 
                                          range_opacity=1,
                                          #range_size = [1,2,3], #used to show the slider range
                                          #split_number=10
                                          #is_show=False,
                                          pos_bottom = '10%'

                                          
                                         ),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False),  # Hide bottom x-axis labels
                             axistick_opts=opts.AxisTickOpts(is_show=False)),  # Hide bottom x-axis ticks

        legend_opts=opts.LegendOpts(is_show=False),
        
    )
)

#on the first run, must run this and render_notebook() separately 
c.load_javascript()

In [ ]:
#on the first run, must run this and load_javascript() separately 
c.render_notebook()